# Closed RAG

In [40]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [41]:
# Configurar la clave de API de OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# Cargar el archivo de texto
loader = TextLoader("movies.txt", encoding="utf-8")
documents = loader.load()

# Dividir el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Initialize embeddings (e.g., OpenAI)
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_KEY"),
    model="text-embedding-3-small"
)

# Store embeddings in vector database (Chroma in-memory DB)
vectorstore = Chroma.from_documents(docs, embeddings)

In [42]:
# Define a custom prompt template that explicitly restricts the model
prompt_template = """You are an AI assistant that only knows information from the provided context. 
If information is not present in the context, respond with "No tengo información sobre esto en mis documentos" or "No puedo responder a esta pregunta basado en la información disponible".
Do not use any prior knowledge.

Context: {context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [43]:
# Configurar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# Crear la cadena de preguntas y respuestas
# qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" puts all retrieved docs in context at once
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10}),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    }
)

In [44]:
# Realizar una consulta
query = "¿Dame los titulos de peliculas que conozcas?"
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result)

Pregunta: ¿Dame los titulos de peliculas que conozcas?
Respuesta: {'query': '¿Dame los titulos de peliculas que conozcas?', 'result': '- Inception (2010)\n- The Godfather (1972)\n- Interstellar (2014)\n- The Shawshank Redemption (1994)', 'source_documents': [Document(metadata={'source': 'movies.txt'}, page_content='⸻\n\nTitle: Inception (2010)\nMovie Code: MOV-003'), Document(metadata={'source': 'movies.txt'}, page_content='⸻\n\nTitle: The Godfather (1972)\nMovie Code: MOV-002'), Document(metadata={'source': 'movies.txt'}, page_content='Actors:\n\t•\tMarlon Brando (Vito Corleone)\n\t•\tAl Pacino (Michael Corleone)\n\t•\tJames Caan (Sonny Corleone)\n\t•\tRobert Duvall (Tom Hagen)\n\t•\tDiane Keaton (Kay Adams)\n\n---\n\nTitle: Inception (2010)\nMovie Code: MOV-003'), Document(metadata={'source': 'movies.txt'}, page_content='⸻\n\nTitle: Interstellar (2014)\nMovie Code: MOV-004'), Document(metadata={'source': 'movies.txt'}, page_content='Actors:\n\t•\tMarlon Brando (Vito Corleone)\n\t•\tA

In [45]:
# Realizar una consulta
query = "¿Que conoces de 'The Godfather'? Si es asi dame detalles de ella."
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result["result"])

Pregunta: ¿Que conoces de 'The Godfather'? Si es asi dame detalles de ella.
Respuesta: Francis Ford Coppola’s iconic crime epic, "The Godfather" (1972), is based on Mario Puzo’s novel and delves into the compelling saga of the Corleone crime family. It portrays the patriarch, Vito Corleone, whose power and influence permeate the underworld of post-war America. No tengo información sobre los actores correctos en mis documentos.


In [46]:
# Realizar una consulta
query = "¿Cuándo se estrenó la primera película de Star Wars?"
result = qa_chain.invoke(query)

print("Pregunta:", query)
print("Respuesta:", result["result"])

Pregunta: ¿Cuándo se estrenó la primera película de Star Wars?
Respuesta: No tengo información sobre esto en mis documentos.
